# 6. 引用对象、可变性和垃圾回收

## 6.2 变量不是盒子（可以理解成贴标签）

Python 变量类似于 Java 中的<mark>引用式变量</mark>, 可以理解为附加在对象上的标注

比如下面的代码，变量 a 和 b 引用同一个列表，而不是那个列表的副本

In [1]:
a = [1, 2, 3]   # *1
b = a           # *2
a.append(4)     # *3
print(b)        # *4

[1, 2, 3, 4]


*1 创建列表 [1, 2, 3], 绑定变量 a
*2 变量 b 绑定 a 引用的值
*3 修改 a 引用的列表，追加一项
*4 如果觉得 b 是一个盒子，存储 a 中 [1, 2, 3] 的副本，那么就说不通了

![将变量看成便利贴](./assets/ch06/var.png)

<span style="color: green"> `b = a` 语句不是把 a 盒子中的内容复制到 b 盒子中，而是在标注为 a 的对象上再贴一个标注 b</span>

对于引用式变量，应该说 <mark>把变量分配给对象更合理</mark>，反过来说就有问题，毕竟，<mark>对象在赋值前就创建了，赋值语句的右边先执行</mark>

在 Python 中，赋值语句 `x = ...` 把名称 x 绑定到右边创建或引用的对象上，在绑定名称之前，对象必须存在

In [2]:
class Gizmo:
    def __init__(self) -> str:
        print(f"Gizmo id: {id(self)}")
        
x = Gizmo()
y = Gizmo() * 10

Gizmo id: 4386563024
Gizmo id: 4386763984


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [3]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'open',
 'quit',
 'x']

在乘法运算中使用 Gizmo 实例会抛出异常，但是还是会打印 y 的 ID，所以表明在尝试求积之前其实会创建一个新的 Gizmo 实例

但是没有创建变量 y，因为在求解赋值语句的右边时抛出了异常

<span style="color: green"> 为了理解 Python 中的赋值语句，应该始终先读右边。对象先走右边创建或获取，然后左边的变量才会绑定到对象上，就像给对象贴上标签一样 </span>

## 6.3 同一性、相等性和别名

In [5]:
charles = {'name': 'Charles', 'age': 30}
lewis = charles # *1

print(lewis is charles)

print(id(charles), id(lewis)) # *2

lewis['balance'] = 1000 # *3
print(charles)

True
4392347392 4392347392
{'name': 'Charles', 'age': 30, 'balance': 1000}


*1 lewis 是 charles 的别名

*2 `is` 和 `id` 确认了这一点

*3 向 lewis 中添加一项相当于向 charles 中添加一项

如果有一个冒充者变量，与上面的变量具有相同的内容

In [6]:
alex = {'name': 'Charles', 'age': 30, 'balance': 1000}  # *1

print(alex == charles)  # *2
print(alex is not charles)  # *3

True
True


经过比较，alex 和 charles 相等，但 alex 绝对不是 charles

*1 alex 指代的对象与分配给 charles 的对象内容一样

*2 比较两个对象，结果相等，<mark>这是因为 dict 类的 `__eq__` 方法就是这样实现的 </mark>

*3 但它们是不同的对象，在 Python 中，使用 `a is not b` 判断两个对象的标识是否不通

lewis 和 charles 是别名，两个变量绑定同一个对象，而 alex 不是 charles的别名，因为二者绑定的是不同的对象

alex 和 charles 绑定的对象具有相同的值，但是它们的标识不同

> - id() 最常用于调试
>
> - id()函数在实际编程中很少使用，对象的标识最常使用 is 运算符比较，无须直接调用 id() 函数

> `==` 运算符比较两个对象的值，而 `is` 比较对象的标识
>
> 编程时，通常关注的是值，而不是标识，所以在 Python 中 `==` 出现的频率更高
>
> 比较一个变量和一个单例时，应该使用 `is`。目前最常使用 is 检查变量绑定的值是不是 `None`

`is` 运算符比 `==` 速度快，因为不能重载，直接比较两个整数 ID

`a == b` 其实是语法糖，等同于 `a.__eq__(b)`，继承自 `object` 的 `__eq__` 方法比较两个对象的ID

元组和多数 Python 容器（列表、字典、集合等）一样，存储的是对象的引用，若引用的项是可变的，<mark> 即使是元组本身不可变，项依旧可以更改 </mark>

<span style="color: green"> 元组的不可变其实是指 tuple 数据结构的物理内容（存储的引用）不可变，与引用的对象无关 </span>

In [7]:
t1 = (1, 2, [30, 40])       # *1
t2 = (1, 2, [30, 40])       # *2
print(t1 == t2)             # *3
print(id(t1), id(t2))

print(id(t1[-1]))           # *4
t1[-1].append(99)           # *5
print(t1)

print(id(t1[-1]))           # *6

print(t1 == t2)             # *7

True
4392167936 4392160128
4392096576
(1, 2, [30, 40, 99])
4392096576
False


*1 t1 不可变，但是 t1[-1] 可变

*2 构建元组 t2, 所含的项与 t1 一样

*3 虽然 t1 和 t2 是不同的对象，但是二者相等

*4 t1[-1] 的 ID 是 4392096576

*5 就地修改 t1[-1]

*6 t1[-1]的标识没变，只是值变了

*7 t1 和 t2 不相等

## 6.4 默认做浅拷贝

<mark>复制列表（或多数内置的可变容器）最简单的方式是使用内置的类型构造函数</mark>

In [8]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
print(l2)
print(l1 == l2)
print(l1 is l2)

[3, [55, 44], (7, 8, 9)]
True
False


`list(l1)` 创建 l1 的副本

虽然两者的值相等，但是二者指代不同的对象

<span style="color: green"> 对列表和其他可变序列来说，还可以使用简洁的 `l2 = l1[:]` 语句创建副本 </span>

构造函数或者 [:] 做的是浅拷贝

- <mark> 浅拷贝</mark>：复制最外层容器，副本中的项是源容器中项的引用
    - 若所有的项都是不可变的，那么这种行为没问题，还能节省内存
    - 若有可变项，可能存在问题

具体的代码可以点击[此链接](https://pythontutor.com/render.html#code=l1%20%3D%20%5B3,%20%5B66,%2055,%2044%5D,%20%287,%208,%209%29%5D%0Al2%20%3D%20l1%5B%3A%5D%0A%0Al1.append%28100%29%0Al1%5B1%5D.remove%2855%29%0A%0Aprint%28f'l1%3A%20%7Bl1%7D'%29%0Aprint%28f'l2%3A%20%7Bl2%7D'%29%0A%0Al2%5B1%5D%20%2B%3D%20%5B33,%2022%5D%0Al2%5B2%5D%20%2B%3D%20%2810,%2011%29%0A%0Aprint%28f'l1%3A%20%7Bl1%7D'%29%0Aprint%28f'l2%3A%20%7Bl2%7D'%29&cumulative=false&curInstr=10&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=311&rawInputLstJSON=%5B%5D&textReferences=false)

![copy](./assets/ch06/copy.png)

l2 是 l1 的浅拷贝，只拷贝了最外层，所以对 l1[1] 进行操作会对 l2 产生影响，因为两者绑定的都是同一个对象

- 对可变的对象来说，比如引用的列表 `+=` 运算符就地修改列表
- 对于不可变对象，如元组，`+=`运算符创建一个新的元组，重新绑定变量

`copy` 模块提供 `copy` 和 `deepcopy` 函数分别为任意对象做浅拷贝和深拷贝

## 6.5 函数的参数是引用时

Python 唯一支持的参数传递模式是<mark>共享传参</mark>

- 共享传参指的是函数的形参获得实参引用的副本
- 函数内部的形参是实参的别名
- 所以，函数可能会修改作为参数传入的可变对象，但是不能修改对象的标识

In [9]:
def f(a, b):
    a += b
    return a

x, y = 1, 2

print(f(x, y))
print(x, y)     # *1

a = [1, 2]
b = [3, 4]
print(f(a, b))
print(a, b)     # *2

t = (10, 20)
u = (30, 40)
print(f(t, u))  # *3

3
1 2
[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]
(10, 20, 30, 40)


*1 数值 x 没变

*2 列表 a 变了

*3 元组 t 没变

### 不要使用可变类型作为参数的默认值

<mark>可选参数可以有默认值，但是应该避免使用可变的对象作为参数的默认值</mark>

In [11]:
class HauntedBus:
    def __init__(self, passengers=[]):      # *1
        self.passengers = passengers        # *2
    
    def pick(self, name):
        self.passengers.append(name)       # *3
        
    def drop(self, name):
        self.passengers.remove(name)

*1  若没有传入 passengers 参数，则绑定默认的列表对象

*2 `self.passengers` 是 `passengers` 的别名，没有提供 passengers 参数时，passengers 是默认列表的别名

*3 调用 `.remove` 和 `.append` 方法，修改的其实是默认列表

**一个诡异的行为**

In [12]:
bus1 = HauntedBus(['Alice', 'Bob'])     # *1
print(bus1.passengers)

bus1.pick('Charlie')
bus1.drop('Alice')
print(bus1.passengers)      # *2

bus2 = HauntedBus()         # *3
bus2.pick('Dan')
print(bus2.passengers)

bus3 = HauntedBus()         # *4
print(bus3.passengers)      # *5
bus3.pick('Eve')

print(bus2.passengers)      # *6

print(bus3.passengers is bus2.passengers)   # *7
print(bus1.passengers)      # *8

['Alice', 'Bob']
['Bob', 'Charlie']
['Dan']
['Dan']
['Dan', 'Eve']
True


*1 一开始，bus1 有两个乘客

*2 行为是正常的

*3 bus2 是空的，使用默认值，分配了一个空列表

*4 bus3 一开始也是空的，分配默认列表

*5 bus3 不为空！应该出现在 bus2 的 Dan 出现在了 bus3 上

*6 登上了 bus3 的 Eve 出现在了 bus2 上

*7 bus2.passengers 和 bus3.passengers 是同一个列表

*8 但 bus1.passengers 是不同的列表

<mark> 问题是，没有指定初始化乘客的 HauntedBus 实例共享了同一个乘客列表

这种问题很难发现，如果传入了参数，则一切正常，否则，就会出现问题。

若没传入参数，默认值就会成为函数对象的属性，当默认值是可变对象，并且修改了它的值，那么后续的函数调用都会受到影响

<span style="color: green"> 可变默认值导致的问题说明了为什么通常使用 None 作为接收可变值的默认参数的默认值 </sapn>

可以进行如下的修改

In [ ]:
class HauntedBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

### 防御可变参数

若定义的函数接收可变可变参数，应该谨慎考虑调用方是否期望修改传入的参数

比如：若函数接收一个字典，而且在处理的过程中要修改它，那么这个副作用要不要体现在函数外部？

In [ ]:
class TwilightBus:
    """让乘客销声匿迹的校车"""
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []            # *1
        else:
            self.passengers = passengers    # *2
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)        # *3

*1 当 passengers 为 None，创建一个新列表

*2 `self.passengers = passengers` 这个赋值语句把 `self.passengers` 变成了 `passengers` 的别名，而后者是传给 `__init__` 方法的实参的别名

*3 在 `self.passengers` 上调用 `.remove()` 和 `.append()` 方法，其实会修改传给构造函数的列表

正确做法是：校车自己维护乘客列表

In [ ]:
class TwilightBus:
    """让乘客销声匿迹的校车"""
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []            
        else:
            self.passengers = list(passengers)  # *1

*1 创建 passengers 列表的副本，如果不是列表，就把它转换为列表

这种方式不会影响初始化校车传入的参数，而且还有一个好处是更加灵活
- 传给 passengers 参数的值可以是元组hove任何其他可迭代对象，如 set，甚至数据库查询结果，因为 list 构造函数可以接收任何可迭代对象
- 自己创建并管理列表，可以确保 `.remove()` 和 `.append()` 操作正常执行

> 除非方法确实想修改通过参数传入的对象，否则在类中直接把参数赋值给实例变量之前一定要三思
>
> 如果不确定，那么就创建副本，虽然会消耗一定的 CPU 和内存，但是与速度和资源相比，难以察觉的 bug 更是严重的问题

## del 和垃圾回收

del 是函数而不是语句

`del x` 和 `del(x)` 一样仅仅是因为在 Python 中，`x` 和 `(x)` 表达式往往是一样的

del 删除引用，而不是对象，del 可能导致对象被当作垃圾回收

特殊方法 `__del__` 不负责销毁实例，而且不应该在代码中调用

`__del__` 方法不好实现，往往出力不讨好

> A. Jesse Jiryu Davis 写的“PyPy, Garbage Collection, and a Deadlock”一文对 __del__ 方法的恰当用法和不当用法做了讨论

## Python 对不可变类型施加的把戏

...(略过

## 延伸阅读

> Wesley Chun 在 EuroPython 2011 会上所做的演讲“Understanding Python's Memory Model, Mutability, and Methods”不仅涵盖了本章的主题，还讨论了特殊方法的使用
>
> 在《Python 开发者指南》中，Pablo Galindo 写的“Design of CPython's Garbage Collector”一文深入探讨了 Python 的垃圾回收机制，方便不同层次的贡献者了解 CPython 的实现，无论是新手还是老手